###Databricks notebook to run a deep learning CNN model to classify fashion items

This notebook steps through how to implement an image recognition model using a convolutional network. It starts with a basic CNN architecture to classify fashion from the Fashion MNIST data set found here: https://github.com/zalandoresearch/fashion-mnist. The data set consists of images (28*28 in size) in 10 different categories. There are 60,000 images for training and 10,000 images for testing. There are optional steps in the notebook to demonstrate use of more advanced network features. 
    
The model is built using Tensorflow and Keras and assumes you are running on a Databricks cluster and machine learning libraries have been pre-installed. 

This notebook is adapted from work published here: 
   (1) https://github.com/amynic/azureml-sdk-fashion and 
   (2) https://github.com/Microsoft/CNTK/tree/v2.0/Tutorials.

### Import core libraries and specify tensorflow and keras

In [3]:
import tensorflow as tf
import os
import time
os.environ["TF_CPP_MIN_LOG_LEVEL"]= "2"
print("tensorflow Version is: " + str(tf.__version__))

import numpy as np
os.environ['KERAS_BACKEND'] = 'tensorflow'
from keras import backend as K
print(os.environ['KERAS_BACKEND'])

In [4]:
# Import the required Keras functions that will be used to create the CNN
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import utils, losses, optimizers
import matplotlib.pyplot as plt

### Load the Fashion MNIST data set and visualize an image

The fasion MNIST data set is included with the Keras library so we need to pull it into a training and test datasets. Each image is associated with a label from 1 of 10 classifications: 
  * Label 0: T-shirt/top
  * Label 1: Trouser 
  * Label 2: Pullover 
  * Label 3: Dress 
  * Label 4: Coat 
  * Label 5: Sandal 
  * Label 6: Shirt 
  * Label 7: Sneaker 
  * Label 8: Bag 
  * Label 9: Ankle boot

In [6]:
# Data for training and testing
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

print(x_train.shape, 'train set')
print(x_test.shape, 'test set')

# Define the text labels
fashion_mnist_labels = ["Top",          # index 0
                        "Trouser",      # index 1
                        "Jumper",       # index 2 
                        "Dress",        # index 3 
                        "Coat",         # index 4
                        "Sandal",       # index 5
                        "Shirt",        # index 6 
                        "Trainer",      # index 7 
                        "Bag",          # index 8 
                        "Ankle boot"]   # index 9



In [7]:
# Visualize an image. To visualize a different image, set the index to any value between 0 and 59999
img_index=8

# Display an image from the data set
label_index = y_train[img_index]
plt.imshow(x_train[img_index])
plt.title(fashion_mnist_labels[label_index])
 
display()

### Normalize and prepare the data

The data set is loaded into memory and will be normalized so the pixel values in each image range between 0-1 instead of 0-255 pixels. This will help the model converge faster because it is working with smaller numbers.

In this section we also one-hot encode the labels for each image. This is done so the model does not treat the label (fashion category) with an implied numeric ranking. For instance, T-Shirts are labeled '0' and Sneakers are labeled '7'. We don't want the model to interpret these values with any numeric ranking. An array will be created and each image will be represented in the array with a '1' corresponding to its label. For instance, an image with an array that looks like this: array([0,0,0,0,0,0,0,0,0,1]) means the image label corresponds to the 9th index which is 'ankle boots'.

In [9]:
# Set number of categories
num_classes = 10

# Set image input dimensions 
img_rows,img_cols = 28,28

# Reshape the array without changing the data.  
# Parameters = number of elements in the input array, the new shape (28*28), and the order 
x_train = x_train.reshape(60000, img_rows, img_cols, 1)
x_test = x_test.reshape(10000, img_rows, img_cols, 1)

# Type convert and scale the data
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Save the labels for the confusion matrix later
test_labels = y_test
print(test_labels)

# One-hot encoding of images
# This creates an array equal to the number of labels (fashion categories).  Each image
# is represented in the vector with a 1 corresponding to it's label.  
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)


### Define the CNN network architecture
We will use the sequential API and compile the model with a common algorithm and loss metric.

After running through the experiment, consider coming back to this section and changing some of the parameters, re-run the code, and see how the model and accuracy change.

In [11]:
#First, set some parameters

num_classes = 10

# Sample size that will be processed independently, in parallel
batch_size = 256

#Define the CNN model as a sequential model. Every layer passes data forward to the next layer in the network. 
model = Sequential()

# First convolutional layer uses the kernels/filters to extract features and passes them to the next layer.
model.add(Conv2D(64, kernel_size=(3,3), activation = 'relu', input_shape=(28,28,1)))

# Pooling layer reduces dimensionality, helps to avoid overfitting
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, kernel_size=(3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))

#model.add(Dropout(0.5))

# Add final, fully connected layer.  Image will be classified into one of the fashion classes.  
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=losses.categorical_crossentropy,
              optimizer=optimizers.Adam(),
              metrics=['accuracy'])

# Verify model architecture is as expected
model.summary()


###Train the model
To train the model, use the fit() method in Keras. This code compiles the CNN model, assigns a common optimizer and loss function and identifies the output metrics to be shown.   
A timer is started to show how long the model takes to run. The test data set is passed in as the validation set so we can see how the accuracy differs between the training set and the validation set.

In [13]:
# Train the model and return loss and accuracy for each epoch

# The number of training passes
epochs = 6

# Batch size is the sample size that will be processed independently, in parallel
batch_size = 256

start = time.time()
hist = model.fit(x_train, y_train, 
                  batch_size=batch_size, 
                  epochs=epochs, verbose=1)
end = time.time()

In [14]:
# Print how long it took to train the  model
print('Time to train model (sec): ', (end-start))

### Score the model

In [16]:
# Evaluate the model on the test data
score = model.evaluate(x_test, y_test, verbose=0)
print('Test Loss: ', score[0])
print('Test Accuracy: ', score[1])

###Plot model evaluation metrics

In [18]:
# Plot model accuracy across the epochs 

accuracy = hist.history['acc']

epochs = range(len(accuracy))

fig = plt.figure()
plt.plot(epochs, accuracy, 'bo')
plt.title('Training accuracy')
plt.legend()

display(fig)

In [19]:
# Plot model loss across the epochs

loss = hist.history['loss']

fig = plt.figure()
plt.plot(epochs, loss, 'bo')
plt.title('Training loss')
plt.legend()

display(fig)

### Predict labels for 15 images from test set

In [21]:
# Run this code to see if the labels for each of 15 images was predicted correctly
predictions = model.predict(x_test)

# Plot a random sample of 15 test images, their predicted labels and ground truth
figure = plt.figure(figsize=(20, 8))
for i, index in enumerate(np.random.choice(x_test.shape[0], size=15, replace=False)):
    ax = figure.add_subplot(3, 5, i + 1, xticks=[], yticks=[])
    # Display each image
    ax.imshow(np.squeeze(x_test[index]))
    predict_index = np.argmax(predictions[index])
    true_index = np.argmax(y_test[index])
    # Set the title for each image
    ax.set_title("{} ({})".format(fashion_mnist_labels[predict_index], 
                                  fashion_mnist_labels[true_index]),
                                  color=("green" if predict_index == true_index else "red"))
# Visualize in databricks
display()

###Create and display a confusion matrix

In [23]:
# Import the libraries needed to create the confusion matrix
from sklearn.metrics import confusion_matrix
import itertools

In [24]:
# Create a confusion matrix to see how the predicted labels did compared to the true labels

# Rather than using the explicit probabilities generated by the prediction method,
# round the numbers  
rounded_predictions = model.predict_classes(x_test)

#print(rounded_predictions)
#print(test_labels)

# Test labels were created in the data pre-processing step 
cm = confusion_matrix(test_labels, rounded_predictions)

In [25]:
# Confusion matrix code borrowed from here: https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

def plot_confusion_matrix( cm, classes,
                           normalize=False,
                           title='Confusion matrix',
                           cmap=plt.cm.Blues):
  
  # This function prints and plots the confusion matrix
 
    plt.figure()
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
 
    # If normalize, plot the raw numbers, otherwise plot the rounded numbers 
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print('Normlaized confusion matrix')
    else:
        print("Confusion matrix, without normalization")
   
    # Text prints out in white the number of true predictions and
    # black for the number of false predictions  
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
      plt.text(j, i, cm[i,j],
        horizontalalignment="center",
        color="white" if cm[i,j] > thresh else "black")
             
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')           
      
 
cm_plot_labels = fashion_mnist_labels
plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix')
display()                           

###Show the impact of changing the activation function

To easily see the impact of changing the activation function, a python function is defined below to construct, compile, train, and score a model.

In [27]:
def runCNN(activation, verbose):

  img_rows, img_cols = 28, 28
  input_shape = (img_rows, img_cols, 1)
  epochs = 1

  # Build the CNN
  model = Sequential()
  
  # Convolution Layer
  model.add(Conv2D(64, kernel_size=(3, 3),
                 activation=activation,
                 input_shape = (28,28,1))) 
  
  # Pooling with stride (2, 2)
  model.add(MaxPooling2D(pool_size=(2, 2)))
  
  model.add(Conv2D(64, kernel_size=(3,3), activation = 'relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))

  # Flatten layer
  model.add(Flatten())
  model.add(Dense(128, activation='relu'))

   # Randomly delete 50% of neurons to avoid overfitting  
  model.add(Dropout(0.5))
  
   # Apply softmax
  model.add(Dense(num_classes, activation='softmax'))
 
  # Loss function (crossentropy) and Optimizer (Adam)
  model.compile(loss = losses.categorical_crossentropy,
              optimizer = optimizers.Adam(),
              metrics=['accuracy'])
 
  # Train model
  model.fit(x_train, y_train, 
                batch_size=batch_size, 
                epochs=epochs, verbose=0)

  # Evaluate model
  score = model.evaluate(x_test, y_test, verbose=0)
  
  # Return
  return score

In [28]:
# Sigmoid activation  

score_sigmoid = runCNN('sigmoid', 0)
print('Sigmoid, Test loss:', score_sigmoid[0])
print('Sigmoid, Test accuracy:', score_sigmoid[1])


In [29]:
# Tanh activation

score_tanh = runCNN('tanh', 0)
print('tanh, Test loss:', score_tanh[0])
print('tanh, Test accuracy:', score_tanh[1])


In [30]:
# Relu activation

score_sigmoid = runCNN('sigmoid', 0)
print('Sigmoid, Test loss:', score_sigmoid[0])
print('Sigmoid, Test accuracy:', score_sigmoid[1])
